In [74]:
from datasets import load_dataset
import pandas as pd
import os
from dotenv import load_dotenv
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import pandas
import json
import re
import numpy as np
import openai
import random
import math

from sklearn.metrics.pairwise import cosine_similarity
from pprint import pprint
from pathlib import Path
from statistics import median


from langchain.embeddings import OpenAIEmbeddings

### Loading datasets

In [2]:
dataset_lit = load_dataset("jerteh/SrpELTeC")

In [3]:
dataset_lit

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 157430
    })
})

In [4]:
lit_large = dataset_lit['train']['text']

In [5]:
print(lit_large[:5])

['', '<s>10 SRPSKA KNjIŽEVNA ZADRUGA 10</s>', '<s>DVA IDOLA</s>', '<s>BOGOBOJ ATANACKOVIĆ</s>', '<s>U BEOGRADU</s>']


In [6]:
lit_large.index("<s>POP ĆIRA I POP SPIRA</s>")

59300

In [7]:
text_lit = lit_large[59300 : 59300+3500]

In [8]:
len(text_lit)

3500

In [9]:
dataset_wiki = load_dataset("jerteh/SrpWiki")

In [10]:
dataset_wiki

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 3818535
    })
})

In [11]:
text_wiki = dataset_wiki['train']['text']
print(text_wiki[:10])
print(len(text_wiki))
print(type(text_wiki))

['UTF-8 UTF-8 varijanta je najzgodnija za kodiranje većinski latiničnog teksta.', 'Dato je i kratko uputstvo za korišćenje te varijante u Microsoft Word-u, Netscape Composer-u i tekstualnom editoru Kate.', 'U tekstu su takođe preporučeni standardni Unicode fontovi koji omogućavaju laku prenosivost teksta sa računara na računar ili za objavljivanje teksta na Internet.', 'Prvi računari su bili pravljeni pretežno za englesko govorno područje i imali su podršku samo za engleski alfabet, za brojeve, zagrade i još po neki kontrolni karakter, što je činilo ukupno 128 mogućih slova (u 7 bita).', 'To je bio tzv. "-{ASCII}-" ili "-{US-ASCII}-" standard.', '1968. godine je skup karaktera proširen na 256 (8 bita), a "gornjih" 128 karaktera je bilo korišćeno za dodatne karaktere.', 'Iz neke navike je i ovaj prošireni "-{ASCII}-" nazivan "-{ASCII}-", tako da tu često dolazi do zabune.', 'Da bi postojala podrška za više jezika, smišljane su tzv. kodne strane ("-{Code Page}-") koje definišu ponašanje 

In [12]:
dataset_news = load_dataset("jerteh/SrpKorNews")

In [13]:
dataset_news

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 337
    })
})

In [14]:
text_news = dataset_news['train']['text']
print(len(text_news))
print(type(text_news))

337
<class 'list'>


In [15]:
dataset_sci = load_dataset("procesaur/STARS")

In [16]:
dataset_sci

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 7559526
    })
})

In [17]:
text_sci = dataset_sci['train']['text']

In [18]:
print(text_sci[:5])
print(len(text_sci))
print(type)

['<s>Uticaj brzine vazduha na aerosole formirane od 1 % emulzija 112 Uticaj brzine vazduha na aerosole formirane od 6 % emulzija 119 Uticaj brzine vazduha na aerosole formirane od 10 % emulzija 128 Strujanje aerosola vode u cevi</s>', '<s>Završna razmatranja strujanja aerosola u cevi formiranog emulzije EO3</s>', '<s>Moguće strukture aerosol čestice NaCl: a) neporozna čvrsta, b) porozna čvrsta, c) čvrsta sa otvorenim džepovima rastovrenog NaCl, d) čvrsta čaura sa vodenim jezgrom, e) čvrsta NaCl čestica sa jezgrom vodenog rastvora NaCl, f) vodena kapljica NaCl.</s><s>Linije na slikama predstavljaju adsorbovanu vodu.</s>', '<s>Atomiziranje centrifugalnom silom kod rotirajućieg atomizer „rotary atomizer“</s>', '<s>Dezintegrisanje tečnog filma u zavisnosti od relativne brzine a) i c) prednji snimak filma b) i d) bočni snimak filma</s>']
7559526
<class 'type'>


### Creating subsets

In [19]:
def create_sentnecs(texts):
    sentences = []
    for text in texts:
        matches = re.findall(r'<s>(.*?)</s>', text)
        sentences.extend(matches)
    
    
    return sentences

In [20]:
def make_subset_from_all_sentences(all_sentences: list[str], final_lenght: int, chunked_lenght:int, random_step: int, random_step_start: int = 2000):
    start = 0
    final_sentences = []
    while start+chunked_lenght < len(all_sentences) and len(final_sentences) < final_lenght:
        final_sentences.extend(all_sentences[start:start+chunked_lenght])
        start += chunked_lenght + random.randint(random_step_start, random_step)
    return final_sentences

In [21]:
final_wiki = make_subset_from_all_sentences(text_wiki, 8000, 30, 10_000)

In [22]:
print(final_wiki[:5])
print(len(final_wiki))

['UTF-8 UTF-8 varijanta je najzgodnija za kodiranje većinski latiničnog teksta.', 'Dato je i kratko uputstvo za korišćenje te varijante u Microsoft Word-u, Netscape Composer-u i tekstualnom editoru Kate.', 'U tekstu su takođe preporučeni standardni Unicode fontovi koji omogućavaju laku prenosivost teksta sa računara na računar ili za objavljivanje teksta na Internet.', 'Prvi računari su bili pravljeni pretežno za englesko govorno područje i imali su podršku samo za engleski alfabet, za brojeve, zagrade i još po neki kontrolni karakter, što je činilo ukupno 128 mogućih slova (u 7 bita).', 'To je bio tzv. "-{ASCII}-" ili "-{US-ASCII}-" standard.']
8010


In [23]:
# sent_news = create_sentnecs(text_news)
# print(sent_news[:5])
# print(len(sent_news))

['Mečka na usijanom limenom krovu', 'Piše: Dušan Vidaković Kada je austrijski konzul u Nici Emil Jelinek 1901. naručio od nemačkog Dajmlera, odjednom, čak 36 automobila, čija je ukupna cena premašila fantastičnih 550 hiljada zlatnih maraka, imao je samo jedan uslov: da sva ta kola nose ime njegove ćerke.', 'Ljudi iz kanstatske firme, naravno, nisu imali ništa protiv, a istorija automobilizma je imala sreću da je Jelinekovoj mezimici kum dao (pogotovo za Špance) vrlo graciozno ime - Mercedes, što na jeziku konkvistadora znaci ljupkost, umilnost, prijatnost.', 'Jelinekova porudžbina značila prekretnicu u poslovanju Dajmlera, a naziv Mercedes je preživeo i fuziju sa fabrikom Karla Benca, 1926. godine, održavši se do danas kao sinonim za kvalitetne, luksuzne i, nadasve, bezbedne automobile svojstvenih vozačima koji ne kriju brige u novčaniku.', 'Međutim, upravo legenda o blistavosti trokrake zvezde doživljava ovih dana ozbiljnu proveru zahvaljujući mercedesu A klase.']
21982152


In [24]:
# final_news = make_subset_from_all_sentences(sent_news, 2000, 30, 100_000)

In [25]:
print(final_news[:5])
print(len(final_news))

['Mečka na usijanom limenom krovu', 'Piše: Dušan Vidaković Kada je austrijski konzul u Nici Emil Jelinek 1901. naručio od nemačkog Dajmlera, odjednom, čak 36 automobila, čija je ukupna cena premašila fantastičnih 550 hiljada zlatnih maraka, imao je samo jedan uslov: da sva ta kola nose ime njegove ćerke.', 'Ljudi iz kanstatske firme, naravno, nisu imali ništa protiv, a istorija automobilizma je imala sreću da je Jelinekovoj mezimici kum dao (pogotovo za Špance) vrlo graciozno ime - Mercedes, što na jeziku konkvistadora znaci ljupkost, umilnost, prijatnost.', 'Jelinekova porudžbina značila prekretnicu u poslovanju Dajmlera, a naziv Mercedes je preživeo i fuziju sa fabrikom Karla Benca, 1926. godine, održavši se do danas kao sinonim za kvalitetne, luksuzne i, nadasve, bezbedne automobile svojstvenih vozačima koji ne kriju brige u novčaniku.', 'Međutim, upravo legenda o blistavosti trokrake zvezde doživljava ovih dana ozbiljnu proveru zahvaljujući mercedesu A klase.']
2010


In [ ]:
# removed_lit = create_sentnecs(text_lit)
# print(removed_lit[:5])
# # print(len(removed_lit))A

In [ ]:
# final_lit = make_subset_from_all_sentences(removed_lit, 1250, 30, 100, random_step_start=0)

In [ ]:
# print(final_lit[:30])
# print(len(final_lit))

In [ ]:
# removed_sci = create_sentnecs(text_sci[:450_000])
# print(removed_sci[:5])
# print(len(removed_sci))

In [ ]:
# final_sci=make_subset_from_all_sentences(removed_sci, 1250, 30, 10_000)
# print(final_sci)
# print(len(final_sci))

### Creating sentences

In [26]:
# crate one function with create_sentences and flag for loop if dataset isWiki? 
def return_dic(sentences):
    sent_lst_dic = [{'sentence': x, 'id' : i} for i, x in enumerate(sentences)]
    return sent_lst_dic

In [152]:
sent_wiki = return_dic(final_wiki)
pprint(sent_wiki[:3])
print(len(sent_wiki))

[{'id': 0,
  'sentence': 'UTF-8 UTF-8 varijanta je najzgodnija za kodiranje većinski '
              'latiničnog teksta.'},
 {'id': 1,
  'sentence': 'Dato je i kratko uputstvo za korišćenje te varijante u '
              'Microsoft Word-u, Netscape Composer-u i tekstualnom editoru '
              'Kate.'},
 {'id': 2,
  'sentence': 'U tekstu su takođe preporučeni standardni Unicode fontovi koji '
              'omogućavaju laku prenosivost teksta sa računara na računar ili '
              'za objavljivanje teksta na Internet.'}]
8010


In [33]:
# sent_news = return_dic(final_news)
# pprint(sent_news[:3])
# print(len(sent_news))

In [ ]:
# sent_sci = return_dic(final_sci)
# pprint(sent_sci[:3])
# print(len(sent_sci))

In [ ]:
# sent_lit = return_dic(final_lit)
# pprint(sent_lit[:10])
# print(len(sent_lit))

In [ ]:
# print(sum(len(sent)))

### Creating combined sentences

Source: https://github.com/FullStackRetrieval-com/RetrievalTutorials/blob/8a30b5710b3dd99ef2239fb60c7b54bc38d3613d/tutorials/LevelsOfTextSplitting/5_Levels_Of_Text_Splitting.ipynb

In [147]:
openai.api_key = os.getenv("OPENAI_API_KEY")
oaiembeds = OpenAIEmbeddings(model='text-embedding-3-small')

In [130]:
buffer_size = 2

In [131]:
def combine_sentences(sentences, buffer_size=buffer_size):
    for i in range(len(sentences)):
        combined_sentence = ''

        for j in range(i - buffer_size, i):
            if j >= 0:
                combined_sentence += sentences[j]['sentence'] + ' '
        combined_sentence += sentences[i]['sentence']

        for j in range(i + 1, i + 1 + buffer_size):
            if j < len(sentences):
                combined_sentence += ' ' + sentences[j]['sentence']

        sentences[i]['combined_sentence'] = combined_sentence

    return sentences

In [148]:
def generate_embeddings(sentences):
    embeddings = oaiembeds.embed_documents([x['combined_sentence'] for x in sentences])
    for i, sentence in enumerate(sentences):
        sentence['combined_sentence_embedding'] = embeddings[i]
    
    return sentences

In [133]:
def calculate_cosine_distances(sentences):
    distances = []
    
    for i in range(len(sentences) - 1):
        embedding_current = sentences[i]['combined_sentence_embedding']
        embedding_next = sentences[i + 1]['combined_sentence_embedding']
        similarity = cosine_similarity([embedding_current], [embedding_next])[0][0]
        distance = 1 - similarity
        distances.append(distance)

        sentences[i]['distance_to_next'] = distance

    # Optionally handle the last sentence
    # sentences[-1]['distance_to_next'] = None  # or a default value

    return distances, sentences

In [134]:
def get_breakpoint(sentences, distances, threshold):

    breakpoint_distance_threshold = np.percentile(distances, threshold) # If you want more chunks, lower the percentile cutoff
    indices_above_thresh = [i for i, x in enumerate(distances) if x > breakpoint_distance_threshold]
    start_index = 0
    chunks = []

    for index in indices_above_thresh:
        end_index = index
        group = sentences[start_index:end_index + 1]
        combined_text = ' '.join([d['sentence'] for d in group])
        chunks.append(combined_text)
        
        start_index = index + 1

    if start_index < len(sentences):
        combined_text = ' '.join([d['sentence'] for d in sentences[start_index:]])
        chunks.append(combined_text)

    return chunks

In [137]:
combined_wiki = combine_sentences(sent_wiki)
pprint(combined_wiki[:3])
print(len(combined_wiki))

[{'combined_sentence': 'UTF-8 UTF-8 varijanta je najzgodnija za kodiranje '
                       'većinski latiničnog teksta. Dato je i kratko uputstvo '
                       'za korišćenje te varijante u Microsoft Word-u, '
                       'Netscape Composer-u i tekstualnom editoru Kate. U '
                       'tekstu su takođe preporučeni standardni Unicode '
                       'fontovi koji omogućavaju laku prenosivost teksta sa '
                       'računara na računar ili za objavljivanje teksta na '
                       'Internet.',
  'id': 0,
  'sentence': 'UTF-8 UTF-8 varijanta je najzgodnija za kodiranje većinski '
              'latiničnog teksta.'},
 {'combined_sentence': 'UTF-8 UTF-8 varijanta je najzgodnija za kodiranje '
                       'većinski latiničnog teksta. Dato je i kratko uputstvo '
                       'za korišćenje te varijante u Microsoft Word-u, '
                       'Netscape Composer-u i tekstualnom editoru Kate. U '
 

In [138]:
embeddings_wiki = generate_embeddings(combined_wiki)
print(embeddings_wiki[0])
print(len(embeddings_wiki))

{'sentence': 'UTF-8 UTF-8 varijanta je najzgodnija za kodiranje većinski latiničnog teksta.', 'id': 0, 'combined_sentence': 'UTF-8 UTF-8 varijanta je najzgodnija za kodiranje većinski latiničnog teksta. Dato je i kratko uputstvo za korišćenje te varijante u Microsoft Word-u, Netscape Composer-u i tekstualnom editoru Kate. U tekstu su takođe preporučeni standardni Unicode fontovi koji omogućavaju laku prenosivost teksta sa računara na računar ili za objavljivanje teksta na Internet.', 'combined_sentence_embedding': [-0.024897929199118405, 0.014086221298445854, 0.02433603062673318, -0.0024098677015776237, 0.014105596533224438, -0.01677945995773277, -0.00382188054407459, -0.01685696275949225, 0.0064957435029216375, 0.004301432040344406, -0.011974256446322747, 0.04576568662971187, -0.02681613518419767, 0.014357482967249174, 0.03925541030372244, 0.03146633122944359, -0.040107946711012143, -0.05940626542105686, -0.042549300798919464, -0.03510898712652599, -0.013931214763604323, -0.0274942888

In [126]:
distances_wiki, sentences_wiki  = calculate_cosine_distances(embeddings_wiki)
print(distances_wiki[:10])
print(len(distances_wiki))

[0.011682990539164773, 0.006656105841141624, 0.027201744202038913, 0.014940531287105374, 0.01995167430927869, 0.030470513869198967, 0.015295233281451548, 0.038741859654103594, 0.022708416698471456, 0.019575991775448887]
8009


In [42]:
# breakpoint_wiki = get_breakpoint(sentences_wiki, distances_wiki, 65)
# print(breakpoint_wiki[:2])
# print(len(breakpoint_wiki))

['UTF-8 UTF-8 varijanta je najzgodnija za kodiranje većinski latiničnog teksta. Dato je i kratko uputstvo za korišćenje te varijante u Microsoft Word-u, Netscape Composer-u i tekstualnom editoru Kate. U tekstu su takođe preporučeni standardni Unicode fontovi koji omogućavaju laku prenosivost teksta sa računara na računar ili za objavljivanje teksta na Internet. Prvi računari su bili pravljeni pretežno za englesko govorno područje i imali su podršku samo za engleski alfabet, za brojeve, zagrade i još po neki kontrolni karakter, što je činilo ukupno 128 mogućih slova (u 7 bita). To je bio tzv. "-{ASCII}-" ili "-{US-ASCII}-" standard. 1968. godine je skup karaktera proširen na 256 (8 bita), a "gornjih" 128 karaktera je bilo korišćeno za dodatne karaktere. Iz neke navike je i ovaj prošireni "-{ASCII}-" nazivan "-{ASCII}-", tako da tu često dolazi do zabune.', 'Da bi postojala podrška za više jezika, smišljane su tzv. kodne strane ("-{Code Page}-") koje definišu ponašanje tog dodatnog skupa

In [43]:
# type(breakpoint_wiki)

list

In [167]:
import tiktoken
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens
num_tokens_from_string("tiktoken is great!", "cl100k_base")

6

In [168]:
def get_threshold(sentences, distances):
    breakpoints = []
    for threshold in range(40, 100, 5):
        breaks = get_breakpoint(sentences, distances, threshold)
        
        chunk_lengths = []
        for chunk in breaks:
            tokens = num_tokens_from_string(chunk, 'cl100k_base')  # Use a tokenizer instead?
            chunk_lengths.append(tokens)
        
        num_chunks = len(breaks)
        longest_chunk = max(chunk_lengths)
        shortest_chunk = min(chunk_lengths)
        median_length = median(chunk_lengths)

        longest_chunk_indexes = [i for i, length in enumerate(chunk_lengths) if length == longest_chunk]
        shortest_chunk_indexes = [i for i, length in enumerate(chunk_lengths) if length == shortest_chunk]
        median_chunk_indexes = [i for i, length in enumerate(chunk_lengths) if length == median_length]

        if median_length > 90:
            return threshold
        # breakpoints.append({
        #     'threshold': threshold,
        #     'chunk': breaks,
        #     'chunk_lengths': chunk_lengths,
        #     'num_chunks': num_chunks,
        #     'longest_chunk': longest_chunk,
        #     'shortest_chunk': shortest_chunk,
        #     'median_length': median_length,
        #     'longest_chunk_indexes': longest_chunk_indexes,
        #     'shortest_chunk_indexes': shortest_chunk_indexes,
        #     'median_chunk_indexes': median_chunk_indexes

        # })

    return 90

Threshold: 40
  Number of chunks: 4806
  Longest chunk length: 900 tokens (Indexes: [2556])
  Shortest chunk length: 3 tokens (Indexes: [3060])
  Median chunk length: 51.0 tokens
  Median chunk length: 51.0 tokens (Indexes: [19, 82, 210, 341, 452, 471, 476, 483, 724, 828, 1041, 1050, 1057, 1093, 1210, 1252, 1407, 1435, 1505, 1510, 1605, 1940, 1995, 2048, 2081, 2238, 2245, 2550, 2689, 2815, 2861, 2887, 2973, 3003, 3074, 3241, 3461, 3517, 3558, 3590, 3643, 3746, 3815, 3823, 3826, 3835, 3837, 3858, 3865, 3887, 3893, 3933, 3939, 3945, 3948, 3954, 3956, 4070, 4075, 4080, 4106, 4125, 4322, 4378, 4419, 4464, 4527, 4624])
Threshold: 45
  Number of chunks: 4406
  Longest chunk length: 948 tokens (Indexes: [2336])
  Shortest chunk length: 3 tokens (Indexes: [2798])
  Median chunk length: 55.0 tokens
  Median chunk length: 55.0 tokens (Indexes: [265, 401, 682, 754, 874, 1200, 1295, 1478, 1673, 1760, 2052, 2444, 2491, 2525, 2584, 2711, 2751, 2823, 2955, 3267, 3415, 3501, 3926, 3946, 4083, 4230, 42

In [169]:
chunks_wiki = get_breakpoint(sentences=sent_wiki, distances=distances_wiki, threshold=70)
pprint(chunks_wiki[0])
print(type(chunks_wiki[0]))
print(len(chunks_wiki))

('UTF-8 UTF-8 varijanta je najzgodnija za kodiranje većinski latiničnog '
 'teksta. Dato je i kratko uputstvo za korišćenje te varijante u Microsoft '
 'Word-u, Netscape Composer-u i tekstualnom editoru Kate. U tekstu su takođe '
 'preporučeni standardni Unicode fontovi koji omogućavaju laku prenosivost '
 'teksta sa računara na računar ili za objavljivanje teksta na Internet. Prvi '
 'računari su bili pravljeni pretežno za englesko govorno područje i imali su '
 'podršku samo za engleski alfabet, za brojeve, zagrade i još po neki '
 'kontrolni karakter, što je činilo ukupno 128 mogućih slova (u 7 bita). To je '
 'bio tzv. "-{ASCII}-" ili "-{US-ASCII}-" standard. 1968. godine je skup '
 'karaktera proširen na 256 (8 bita), a "gornjih" 128 karaktera je bilo '
 'korišćeno za dodatne karaktere. Iz neke navike je i ovaj prošireni '
 '"-{ASCII}-" nazivan "-{ASCII}-", tako da tu često dolazi do zabune. Da bi '
 'postojala podrška za više jezika, smišljane su tzv. kodne strane ("-{Code '
 'Pa

In [185]:
def split_chunk(big_chunk):
    splits = []
    sentences = big_chunk.split(".")
    current_sentence = ""
    for sentence in sentences:
        token_size = num_tokens_from_string(current_sentence, 'cl100k_base')
        if 50 < token_size <450:
            # if current_sentence == '':
            #     print("AAAAAAAAAAAAAAA")
            splits.append(current_sentence)
            current_sentence = ''
        elif token_size>=450:
            current_sentence = ""
        else:
            current_sentence += sentence
    return splits

In [186]:
filthered = []
for chunk in chunks_wiki:
    token_num = num_tokens_from_string(chunk, 'cl100k_base')
    if token_num < 50:
        continue
    if token_num < 450:
        if chunk == '':
            print("AAAAAAAAAAAAAAA")
        filthered.append(chunk)
    else:
        filthered.extend(split_chunk(chunk))

In [187]:
print(len(filthered))
mini = 8298292728282
maxi = 0
suma = 0
tokeni = []
for c in filthered:
    tok = num_tokens_from_string(c, 'cl100k_base')
    tokeni.append(tok)
    mini = min(mini, tok)
    maxi = max(maxi, tok)
    suma += tok
avg = suma/len(filthered)
avg2 = np.mean(tokeni)

print(f"min:{mini}")
print(f"max:{maxi}")
print(f"avg:{avg}")
print(f"avg2:{avg2}")


2347
min:50
max:447
avg:135.43715381337879
avg2:135.43715381337879


### Sentence-by-sentence similarity chunking

In [151]:
# embed the dataset
# get embeddings of the first sentence, measure similarity to the second sentence, concatenate them
# add the third sentence, measure similarity??? to what?
def generate_sentence_embeddings(sentences):
    embeddings = oaiembeds.embed_documents([x['sentence'] for x in sentences])
    for i, sentence in enumerate(sentences):
        sentence['sentence_embedding'] = embeddings[i]
    
    return sentences

In [153]:
wiki_sent_embeddings = generate_sentence_embeddings(sent_wiki)
pprint(wiki_sent_embeddings[0])
print(len(wiki_sent_embeddings))

{'id': 0,
 'sentence': 'UTF-8 UTF-8 varijanta je najzgodnija za kodiranje većinski '
             'latiničnog teksta.',
 'sentence_embedding': [-0.04669330860862333,
                        0.028167136951426627,
                        0.023918575753316563,
                        -0.011438430842743104,
                        0.009584792574986149,
                        0.011070767054097097,
                        0.005183039062929457,
                        -0.026553501434591373,
                        0.006173688715670089,
                        -0.010315013710769191,
                        0.016248700606840125,
                        0.03531615692663298,
                        -0.028841187230610976,
                        0.023653040794850002,
                        0.03838002183201637,
                        0.036970643975540014,
                        -0.007761792490621756,
                        -0.05486361874826923,
                        -0.05012483730265273,
   

In [ ]:
# get sentence embeddings
# get cosine similarity of sentence i and i+1


6

In [166]:
for i in chunks_wiki[:5]:
    print(i)
    print(num_tokens_from_string(i, 'cl100k_base'))
    print('___________________________')
    

UTF-8 UTF-8 varijanta je najzgodnija za kodiranje većinski latiničnog teksta. Dato je i kratko uputstvo za korišćenje te varijante u Microsoft Word-u, Netscape Composer-u i tekstualnom editoru Kate. U tekstu su takođe preporučeni standardni Unicode fontovi koji omogućavaju laku prenosivost teksta sa računara na računar ili za objavljivanje teksta na Internet. Prvi računari su bili pravljeni pretežno za englesko govorno područje i imali su podršku samo za engleski alfabet, za brojeve, zagrade i još po neki kontrolni karakter, što je činilo ukupno 128 mogućih slova (u 7 bita). To je bio tzv. "-{ASCII}-" ili "-{US-ASCII}-" standard. 1968. godine je skup karaktera proširen na 256 (8 bita), a "gornjih" 128 karaktera je bilo korišćeno za dodatne karaktere. Iz neke navike je i ovaj prošireni "-{ASCII}-" nazivan "-{ASCII}-", tako da tu često dolazi do zabune. Da bi postojala podrška za više jezika, smišljane su tzv. kodne strane ("-{Code Page}-") koje definišu ponašanje tog dodatnog skupa slov